<a href="https://colab.research.google.com/github/mjooee/Lietector-/blob/models/word_embedding_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#구글 드라이브 마운트

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#패키지 연결
import os, sys
my_path = '/content/package'
save_path = '/content/drive/MyDrive/Colab Notebooks/package' ## 패키지가 저장될 경로

os.symlink(save_path, my_path)
sys.path.insert(0, my_path)

In [ ]:
#모델 불러오기
from glove import Corpus, Glove

model_dir = '/content/drive/MyDrive'
glove = Glove.load(model_dir + '/glove.model')

In [ ]:
#입력 문서 전처리

import re
import pandas as pd
import matplotlib.pyplot as plt
from konlpy.tag import Okt
from tqdm import tqdm

#토크나이저
okt = Okt()

def preprocessing(input_text):
  removed_data = []

  # 한글 아닌 부분 제거
  s = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", input_text)
  if s != '':
    removed_data.append(s)

  tokenized_data = []
  for sentence in tqdm(removed_data):
      tokenized_sentence = okt.morphs(sentence, stem = True) # 토큰화
  return tokenized_data


In [ ]:
#토큰화된 단어 벡터화
def to_vec(tokenized_data):
  vector_list = []
  for token in tokenized_data:
    if token == []: continue;
    if token in glove.dictionary.keys():
      vector_list.append([token, glove.word_vectors[glove.dictionary[token]]])
  return vector_list

In [ ]:
#입력 문서 평균 벡터 구하기
def get_document_vec(word_vec_list):
  input_vec = None
  if word_vec_list is not None:
    for word_vec in word_vec_list:
      if input_vec is None:
        input_vec = word_vec[1]
      else:
        input_vec = input_vec + word_vec[1]

  if input_vec is not None:
    input_vec = input_vec / vec_cnt

  return input_vec

In [ ]:
#코사인 유사도
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [ ]:
#코사인 유사도 구해서 높은 순서대로 단어와 유사도 출력 -> 단일 문서와 비교 버전
def s_cos_word_doc(word_vec_list, doc_vec):
  answer = []
  for word_vec in word_vec_list:
    answer.append([word_vec[0], cos_sim(doc_vec, word_vec[1])])
  answer = sorted(answer, key = lambda x: x[1], reverse = True)
  return answer

In [ ]:
#코사인 유사도 구해서 높은 순서대로 단어와 유사도 출력 -> 복수 문서와 비교 버전
def m_cos_word_doc(word_vec_list, doc_vec_list):
  answer = []
  is_first = 0
  for doc_vec in doc_vec_list:
    for num, word_vec in enumerate(word_vec_list):
      if is_first == 0:
        answer.append([word_vec[0], cos_sim(doc_vec, word_vec[1])])
      else:
        answer[num][1] += cos_sim(doc_vec, word_vec[1])
    is_first = 1
  answer = sorted(answer, key = lambda x: x[1], reverse = True)

  return answer

In [ ]:
#10위까지 겹치는 단어 없이 (단어, 유사도) 출력
def get_sim_top10(answer, cnt, compare_doc_cnt):
  answerList = []
  num = 0
  i = -1
  while num < 10:
    i += 1
    if i >= cnt: break;
    if answer[i] in answerList:
      continue
    answer[i][1] /= compare_doc_cnt
    answerList.append(answer[i])
    num += 1;
  return answerList

In [ ]:
#비교 문서 벡터 리스트 가져오기

import pickle

compare_vec_list = []

with open("/content/drive/MyDrive/phishing_vec_list.pkl","rb") as f:
    compare_vec_list = pickle.load(f)

#비교 문서 지정 정해야 할 것 -> 일단 다 비교에 사용하고 느리면 변경


In [ ]:
#데이터 받기
input_doc = "자기가 어느 지방 검찰청의 검사라고 소개하더니 빨리 나오지 않으면 징역을 살거라고 겁을 줬다. 그래서 알았다고 했는데 의심되면 인터넷에 검색해보라고도 했다. 내 통장이 범죄의 대포통장으로 쓰였다고 한다. 그러더니 돈을 달라고 요구했다. 계좌 비밀번호를 요구했음"


In [ ]:
#입력 문서 전처리
tokenized_data = preprocessing(input_doc)

#토큰화된 입력문서 벡터화
word_vec_list = to_vec(tokenized_data)

#단어 벡터 수
vec_cnt = len(word_vec_list)

#입력문서 평균 벡터 구하기
input_vec = get_document_vec(word_vec_list)

100%|██████████| 1/1 [00:00<00:00, 17.75it/s]


In [ ]:
is_phishing = 0 #보이스피싱 의심 여부 (0: N, 1: Y)
mean_sim = 0 #평균 유사도

#코사인 유사도 평균 구하기
for doc_vec in compare_vec_list:
  if input_vec is not None:
    doc_sim = cos_sim(input_vec, doc_vec)
  mean_sim += doc_sim

mean_sim /= len(compare_vec_list)

#0.7 이상이면 보이스피싱 의심 단어 상위 10개 출력
if mean_sim >= 0.7:
    is_phishing = 1
    words_sim = m_cos_word_doc(word_vec_list, compare_vec_list)
    words_sim_top10 = get_sim_top10(words_sim, vec_cnt, len(compare_vec_list))
    print(words_sim_top10)

print("보이스피싱 의심 여부:", is_phishing, mean_sim)

[['그렇다', 0.7209344713721564], ['그래서', 0.6944007428715516], ['나오다', 0.6490126707161583], ['알다', 0.6364001012167274], ['되다', 0.6032214697340044], ['않다', 0.5720507428705047], ['돈', 0.5268249159189258], ['계좌', 0.4903354396707888], ['통장', 0.46044993452699884], ['주다', 0.4283504725109588]]
보이스피싱 의심 여부: 1 0.8002348304231182


In [ ]:
#결과 전송

#보내야 할 데이터: 보이스피싱 의심 여부랑 소수점 첫째자리까지 반올림한 유사도, 70%이상이면 의심 단어 10개까지